# Description 
>This shows an example using helios with the Minimum Distortion Embeddings layout from pymde
Helios implements the IPC to enhance pymde allowing non-blocking computation

## Requirements

- networkx
- pymde
- helios

In [1]:
%load_ext autoreload

%autoreload 2
import time
import numpy as np
import networkx as nx


import numpy as np
import pymde
from fury.stream.widget import Widget

from helios import NetworkDraw


## shortest path distortion example

In [7]:
size = 100
s = size
sizes = [s, s, s]
probs = np.array([[0.45, 0.05, 0.02], [0.05, 0.45, 0.07], [0.02, 0.07, 0.40]])
g = nx.stochastic_block_model(sizes, probs, seed=0)
'''
pin = .05
pout = .1
probs = [[pin, pout, 0], [0.0, pin, pout], [pout, 0.0, pin]]
#probs = [[pin, pout, 0], [0.0, pin, pout], [pout, 0.0, pin]]

g = nx.stochastic_block_model(sizes, probs, seed=0, directed=True)
'''
num_nodes = len(g)
edges_list = []
for source, target in g.edges():
    edges_list.append([source, target])
edges_list = np.array(edges_list)


In [8]:


# update edge colors
colors_by_block = [[1, 0, 0], [0, 1, 0,], [0, 0, 1]]
#colors_by_block = [[1, 0, 0, 240], [0, 1, 0, 240], [0, 0, 1, 0]]

edge_colors = []
for source, target in g.edges():
    c0 = colors_by_block[source//s]
    c1 = colors_by_block[target//s]
    edge_colors += [c0, c1]
    
colors_by_block = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
colors = np.array(
    [colors_by_block[i//s] 
     for i in range(len(g))]).astype('float64')
markers = [['o', 's', 'd'][i//s]  for i in range(len(g))]

edge_colors = np.array(edge_colors).astype('float64')
    
centers = np.random.normal(size=(len(g), 3))
node_edge_opacity = .8
node_edge_color = [1, 1, 1]

node_edge_width = node_edge_opacity
node_opacity = node_edge_opacity
network_draw = NetworkDraw(
        positions=centers,
        colors=colors,
        scales=0.5,
        #node_edge_width=.2,
        node_edge_opacity=node_edge_opacity,
        node_opacity=node_opacity,
        node_edge_color=(0, 0, 0),
        #marker=markers,
        marker='3d',
        edge_line_color=edge_colors,
        edge_line_opacity=.1,
        edge_line_width=1,
        #edges=edges_list
)



In [9]:

from fury.stream.widget import Widget
#widget = Widget(showm, encoding='webrtc', port=8777)
widget = Widget(network_draw.showm, ms_stream=0)
widget.start()
time.sleep(2)

0
url: http://localhost:8557?iframe=1&encoding=mjpeg


In [10]:
widget.return_iframe(300)

In [11]:
from helios.layouts import MDE

mde = MDE(
    edges_list, network_draw,
)


In [12]:
mde.start(30, 100, 1, record_positions=False)

In [13]:
mde.stop()
widget.stop()

## cubic 

In [14]:
n_items = 25
edges = pymde.all_edges(n_items).cpu().numpy()
centers = np.random.normal(size=(n_items, 2))
from helios import NetworkDraw

network2 = NetworkDraw(
        positions=centers, 
        scales=.2,
        node_edge_width=0,
        edge_line_color=(0, 0, 0),
        marker='3d',
        window_size=(300, 300),
        edges=edges
    
)

widget2 = Widget(network2.showm, ms_stream=15)
widget2.start()
time.sleep(2)


url: http://localhost:8527?iframe=1&encoding=mjpeg


In [15]:
widget2.return_iframe(200)

In [16]:
from helios.layouts.mde import MDE

mde2 = MDE(
    edges, network2, 
    penalty_name='cubic',
    constraint_name='standardized'
)


In [17]:
# record_positions equals to True means that you
# asking to helios to store and play the layout postions
# Try to run this example with record_positions=False
mde2.start(100, 200, 1, record_positions=True)

In [22]:
widget2.stop()
mde2.stop()

## using parameters in a different penalitie funcion

In [19]:
n_items = 20
edges = pymde.all_edges(n_items).cpu().numpy()
np.delete(edges, [1, 3, 5, 7])
centers = np.random.normal(size=(n_items, 2))
from helios import NetworkDraw

network3 = NetworkDraw(
        positions=centers, 
        scales=.4,
        node_edge_width=0,
        colors=(1, 0,0),
        edge_line_color=(0, 0, 0),
        marker='3d',
        window_size=(300, 300),
        edges=edges
    
)

widget3 = Widget(network3.showm)
widget3.start()
time.sleep(1)


url: http://localhost:8804?iframe=1&encoding=mjpeg


In [20]:

widget3.return_iframe(200)

In [21]:
from helios.layouts.mde import MDE

mde3 = MDE(
    edges, network3, 
    penalty_name='logistic',
    penalty_parameters=[.4, .5],
    constraint_name='standardized'
)


In [22]:
mde3.start(1, 100, 1)

## anchored constrain

In [251]:
%load_ext autoreload

%autoreload 2
import time
import numpy as np
import networkx as nx


import numpy as np
import pymde
from fury.stream.widget import Widget

from helios import NetworkDraw


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [308]:
# from https://github.com/cvxgrp/pymde/blob/main/examples/anchor_constraints.ipynb
depth = 9
n_items = 2**(depth + 1) - 1

edges = []
stack = [0]
while stack:
    root = stack.pop()
    first_child = root*2 + 1
    second_child = root*2 + 2
    if first_child < n_items:
        edges.append([root, first_child])
        stack.append(first_child)
    if second_child < n_items:
        edges.append([root, second_child])
        stack.append(second_child)

# these are the indices of the nodes that we will pin in place
anchors = np.arange(2**depth) + 2**depth - 1


radius = 20

# pin the root to be at (0, 0), and the leaves to be spaced uniformly on a circle
angles = np.linspace(0, 2*np.pi, anchors.shape[0] + 1)[1:]
anchors_pos = radius * np.stack([np.sin(angles), np.cos(angles)], axis=1)
centers = np.random.normal(size=(n_items, 2))*5
centers[anchors] = anchors_pos.copy()


In [309]:
from helios import NetworkDraw
network4 = NetworkDraw(
        positions=centers, 
        scales=.4,
        node_edge_width=0,
        #colors=(1, 0,0),
        edge_line_opacity=.5,
        edge_line_color=(0, 0, 0),
        marker='3d',
        window_size=(500, 500),
        edges=np.array(edges)
    
)

widget4 = Widget(network4.showm, ms_stream=0)
widget4.start()
time.sleep(1)


0
url: http://localhost:8315?iframe=1&encoding=mjpeg


In [310]:
widget4.return_iframe(400)

In [311]:
from helios.layouts import MDE

mde = MDE(
    np.array(edges), network4,
    constraint_name='anchored',
    anchors=anchors.astype('float32'),
    anchors_pos=anchors_pos.astype('float32'),
    use_shortest_path=True
)


In [312]:

mde.start(33, 1000, 1)

In [ ]:
mde.stop()